In [ ]:
%matplotlib inline

# Diffusive Grids

This is just a simple implementation of the idea of 1D adaptive vertical coordinates (Burchard and Beckers, 2004). We take a random profile from WOA13 to give us a stratification, and explore the effect of the stratification, near-surface and background grid diffusivity coefficients.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
import gsw
from scipy.interpolate import interp1d
from scipy.linalg import solve_banded

Load WOA13 and grab the in-situ temperature and practical salinity, as well as coordinate variables.

In [ ]:
temp_d = Dataset('../convert_WOA13/raw/woa13_A5B2_t01_01v2.nc', 'r')
salt_d = Dataset('../convert_WOA13/raw/woa13_A5B2_s01_01v2.nc', 'r')

dep = temp_d.variables['depth'][:]
lat = temp_d.variables['lat'][:]
lon = temp_d.variables['lon'][:]

temp = temp_d.variables['t_an']
salt = salt_d.variables['s_an']

Just choose a nice full-depth (1500m, 57 levels) profile from the Denmark Strait. The observations are annoying because they include the surface and the very bottom, so we'll just use `dep` as our interface locations and average to get observation locations.

In [ ]:
lat_i = lat == 63.5
lon_i = lon == -29.5

temp_c = temp[0,:,lat_i,lon_i].squeeze()
salt_c = salt[0,:,lat_i,lon_i].squeeze()

dep_lay = (dep[1:]    + dep[:-1])    / 2
temp_c  = (temp_c[1:] + temp_c[:-1]) / 2
salt_c  = (salt_c[1:] + salt_c[:-1]) / 2

Use GSW (TEOS-10) to convert to absolute salinity and conservative temperature, then compute the density referenced to 2000m and the buoyancy frequency *on interfaces*.

In [ ]:
sa_c = gsw.SA_from_SP(salt_c, dep_lay, -29.5, 63.5)
ct_c = gsw.CT_from_t(sa_c, temp_c, dep_lay)
r_c  = gsw.rho(sa_c, temp_c, 2000)
n2, dep = gsw.Nsquared(sa_c, ct_c, dep_lay)

Convert from buoyancy frequency to stratification by eliminating gravity and a weird decibar to pascal conversion.

In [ ]:
g = 9.7963
dz_r = np.maximum((n2 * 1e4) / g**2, 0)
k_n2 = dz_r / (r_c[-1] - r_c[0])

Plot our $k^\text{grid}_{N^2}$ diffusivity, on interfaces. We need to interpolate it back to layers so that the diffusion equation applies to interfaces.

In [ ]:
plt.plot(k_n2)

Now we need to interpolate this back to the interfaces located by `dep`.

In [ ]:
f = interp1d(dep, k_n2, bounds_error=False, fill_value="extrapolate")

k_n2 = f(dep_lay)

The discretised implicit diffusion equation is
$$ z_i^{n+1} = z_i^n + I^2 \Delta t^\text{grid}\left(k^\text{grid}_{i+1}\left(z_{i+1}^{n+1} - z_i^{n+1}\right) - k^\text{grid}_i\left(z_i^{n+1} - z_{i-1}^{n+1}\right)\right), $$
which just gives a tridiagonal system. $I$ is the number of layers.

Upper diagonal coefficients are $-I^2 \Delta t k^\text{grid}_{i+1}$, lower diagonal coefficients are similarly $-I^2 \Delta t k^\text{grid}_i$, and diagonal coefficients are simply
$$ 1 + I^2 \Delta t \left(k^\text{grid}_{i+1} + k^\text{grid}_i\right).$$

In [ ]:
k_grid = 0.7 * k_n2 + 0.3 / 1500

dt = 3600 # one hour restoring timescale?
I = dep.size + 1
A = np.zeros((3, I+1))
dep = np.hstack([0, dep, 1500])

A[0,2:]     = -dt * I**2 * k_grid[1:]
A[2,:-2]    = -dt * I**2 * k_grid[:-1]
A[1,[0,-1]] = 1 # boundary conditions
A[1,1:-1]   = 1 + I**2 * dt * (k_grid[1:] + k_grid[:-1])
# skip A[2,-1] (doesn't exist) and A[2,-2] (boundary condition)
dep_next = solve_banded((1, 1), A, dep)

In [ ]:
ax = plt.subplot(121)
plt.plot(dep, '*', label='original')
plt.plot(dep_next, '*', label='iterated')
plt.legend()
ax.invert_yaxis()

ax = plt.subplot(122)
plt.plot(dep_next - dep, '*')